In [ ]:
import concurrent.futures
from datetime import datetime
from playwright.sync_api import sync_playwright
from openpyxl import Workbook
from threading import Lock

# Variables globales
proceso_en_ejecucion = False
estado_codigos = []
total_codigos = 0
codigos_procesados = 0
lock = Lock()  # Para controlar el acceso concurrente a las variables globales

# Función para obtener el estado de un producto, su precio y la cantidad de imágenes usando Playwright
def obtener_estado_y_precio(codigo_padre):
    url_base = f'https://www.marathon.cl/{codigo_padre}.html'
    codigo_padre = codigo_padre  # Inicializar la variable para evitar UnboundLocalError
    with sync_playwright() as p:
        browser = p.chromium.launch(headless=True)
        page = browser.new_page()
        try:
            page.goto(url_base)
            page.wait_for_selector("body", timeout=5000)  # Esperar a que el cuerpo de la página se cargue

            # Verificar si la página redirige al inicio (indicador: URL de redirección)
            if page.url == "https://www.marathon.cl/home/":
                browser.close()
                return codigo_padre, "Web no encontrada", "Precio no disponible", "Cantidad de imágenes no disponible"

            # Buscar los botones de talla y determinar si están seleccionados
            botones_talla = page.query_selector_all("button.size-attribute.swatchable.selectable.swatch-square")
            estado = "Agotado"
            for boton in botones_talla:
                if "selected-assistive-text" in boton.inner_html():
                    estado = "Disponible" if not boton.is_disabled() else "Agotado"
                    break

            # Extraer el precio
            try:
                precio_element = page.query_selector('span.sales > span.value')
                precio = precio_element.inner_text().strip() if precio_element else "Precio no disponible"
            except:
                precio = "Precio no disponible"
            
            # Contar la cantidad de imágenes
            try:
                imagenes = page.query_selector_all('img.galley_img')
                cantidad_imagenes = len(imagenes)
            except:
                cantidad_imagenes = "Cantidad de imágenes no disponible"
            
            browser.close()
            return codigo_padre, estado, precio, cantidad_imagenes
        except Exception as e:
            browser.close()
            return codigo_padre, f"Error: {e}", "Precio no disponible", "Cantidad de imágenes no disponible"

# Función para procesar los códigos y guardar en Excel
def procesar_codigos(codigos):
    global proceso_en_ejecucion, estado_codigos, total_codigos, codigos_procesados
    total_codigos = len(codigos)
    start_time = datetime.now()

    def obtener_estado_concurrente(codigo_padre):
        return obtener_estado_y_precio(codigo_padre)
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:  # Ajustar el número de trabajadores según sea necesario
        futures = {executor.submit(obtener_estado_concurrente, codigo): codigo for codigo in codigos}
        for future in concurrent.futures.as_completed(futures):
            if not proceso_en_ejecucion:
                break
            codigo = futures[future]
            try:
                codigo_padre, estado, precio, cantidad_imagenes = future.result()
            except Exception as exc:
                codigo_padre = codigo  # Inicializar código_padre en caso de error
                estado = f"Error: {exc}"
                precio = "Precio no disponible"
                cantidad_imagenes = "Cantidad de imágenes no disponible"
            with lock:
                estado_codigos.append((codigo_padre, estado, precio, cantidad_imagenes))
                codigos_procesados += 1

            # Actualizar información en la consola
            elapsed_time = datetime.now() - start_time
            tiempo_transcurrido = str(elapsed_time).split('.')[0]  # Formato HH:MM:SS
            print(f"Procesando código {codigos_procesados}/{total_codigos} - Tiempo transcurrido: {tiempo_transcurrido}")

    if proceso_en_ejecucion:
        guardar_resultados()

# Función para pausar el proceso
def pausar_proceso():
    global proceso_en_ejecucion
    proceso_en_ejecucion = False
    print("Proceso pausado. Puede continuar luego.")

# Función para detener el proceso
def detener_proceso():
    global proceso_en_ejecucion
    proceso_en_ejecucion = False
    guardar_resultados()
    print("Proceso detenido.")

# Función para guardar los resultados en Excel
def guardar_resultados():
    global proceso_en_ejecucion
    # Guardar en Excel
    wb = Workbook()
    ws = wb.active
    ws.title = "Control Stock Web"
    ws['A1'] = "CODIGO"
    ws['B1'] = "STATUS WEB"
    ws['C1'] = "PRECIO"
    ws['D1'] = "Cant. Img"
    for i, (codigo, estado, precio, cantidad_imagenes) in enumerate(estado_codigos, start=2):
        ws[f'A{i}'] = codigo
        ws[f'B{i}'] = estado
        ws[f'C{i}'] = precio
        ws[f'D{i}'] = cantidad_imagenes

    # Guardar archivo
    fecha_actual = datetime.now().strftime("%Y-%m-%d")
    nombre_archivo = f"Control Stock Web {fecha_actual}.xlsx"
    wb.save(nombre_archivo)
    print(f"Archivo '{nombre_archivo}' guardado con los estados, precios y cantidad de imágenes de los productos.")
    # Descarga el archivo de Excel en Colab
    from google.colab import files
    files.download(nombre_archivo)

# Función para iniciar el procesamiento
def iniciar_procesamiento(codigos):
    global proceso_en_ejecucion, codigos_procesados, estado_codigos
    if not proceso_en_ejecucion:
        proceso_en_ejecucion = True
        codigos_procesados = 0
        estado_codigos = []
        procesar_codigos(codigos)

# Bloque adicional para ingresar todos los códigos padres
codigos_input = """
10923690001 10727609001 10842374003 10820236005
"""  # Reemplazar con los códigos reales pegados desde un Excel, separados por espacios
codigos = codigos_input.strip().split()

# Iniciar el procesamiento
iniciar_procesamiento(codigos)
